In [106]:
import pandas as pd
import numpy as np

data = pd.read_csv("train.csv")

In [107]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [108]:
data.drop("Loan_ID", axis=1, inplace=True)

Handling Missing values

In [109]:
# Categorical
for col in ['Gender','Married','Dependents','Self_Employed','Credit_History']:
    data[col] = data[col].fillna(data[col].mode()[0])

# Numerical
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].median())
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].median())

Feature Engg

In [110]:
data['TotalIncome'] = data['ApplicantIncome'] + data['CoapplicantIncome']
data['LoanAmount_log'] = np.log(data['LoanAmount'])

#Dropping old columns
data.drop(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount'], axis=1, inplace=True)

In [111]:
X = data.drop('Loan_Status', axis=1)
y = data['Loan_Status']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [115]:
# Convert StringDtype → object BEFORE pipeline
X_train = X_train.copy().astype(object)
X_test = X_test.copy().astype(object)

In [116]:
X = X.astype(object)

categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

In [117]:
X_train.info()

<class 'pandas.DataFrame'>
Index: 491 entries, 154 to 354
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Gender            491 non-null    object
 1   Married           491 non-null    object
 2   Dependents        491 non-null    object
 3   Education         491 non-null    object
 4   Self_Employed     491 non-null    object
 5   Loan_Amount_Term  491 non-null    object
 6   Credit_History    491 non-null    object
 7   Property_Area     491 non-null    object
 8   TotalIncome       491 non-null    object
 9   LoanAmount_log    491 non-null    object
dtypes: object(10)
memory usage: 42.2+ KB


In [118]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [119]:
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int64','float64']).columns.tolist()

In [120]:
# Numerical pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

In [121]:
# Categorical pipeline
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [122]:
# Combine
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

In [133]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=1,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42
)

In [134]:
print(X_train.dtypes)

Gender              object
Married             object
Dependents          object
Education           object
Self_Employed       object
Loan_Amount_Term    object
Credit_History      object
Property_Area       object
TotalIncome         object
LoanAmount_log      object
dtype: object


In [135]:
# Final Pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', rf)
])

model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [136]:
y_pred = model.predict(X_test)

In [137]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8455284552845529


In [131]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 5, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(
    model,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid.fit(X_train, y_train)


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'classifier__max_depth': [None, 5, ...], 'classifier__min_samples_leaf': [1, 2, ...], 'classifier__min_samples_split': [2, 5, ...], 'classifier__n_estimators': [100, 200, ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [138]:
print(grid.best_params_)
print(grid.best_score_)


{'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
0.7983714698000413


In [140]:
import joblib
joblib.dump(model, "loan_model.pkl")

['loan_model.pkl']